In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('../..')
import cortexetl as c_etl
import shared_functions as sf

outdir = "/gpfs/bbp.cscs.ch/project/proj83/home/isbister/spike_sorting_bias_output_data/comparison/"
import os; os.makedirs(outdir, exist_ok=True)

In [ ]:
ground_ma = c_etl.analysis_initial_processing("/gpfs/bbp.cscs.ch/project/proj83/home/isbister/physiology_2023/cortexetl.configs/post_physiology_2023/campaigns/spikesorting_stimulus_tests/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0.yaml") # , loglevel="ERROR"
ground_a = ground_ma.hex0_spikes

# sorted_ma = c_etl.analysis_initial_processing("/gpfs/bbp.cscs.ch/project/proj147/home/isbister/experimental_data/1_silico_spike_sorting/4_etl_converted/8_spikesorting_stimulus_test_neuropixels_13-4-24__8slc_100f_72r_250t_200ms_0/ss/analysis_config_cond_windows.yaml", create_figure_dirs=False) # loglevel="ERROR", 
# sorted_a = sorted_ma.array0

In [ ]:
label = "ground_truth"
a = ground_a
window="evoked_SOZ_100ms"
nc = "ALL"
# n_gid_spikes_plot_tresh = 150
n_gid_spikes_plot_tresh = 100
plot_tresh_prop_non_first = 1.0

# label = "kilosort3"
# a = sorted_a
# window = "Cond0_0_100ms_stim"
# nc = "array0"
# plot_tresh_n_gid_spikes = 50
# plot_tresh_prop_non_first = 1.0

filenames = []
for sim_ind in [0]:
    
    window_by_gid_and_trial = a.features.by_gid_and_trial.df.reset_index().etl.q(simulation_id=sim_ind, window=window, neuron_class=nc)
    spikes_df = a.repo.report.df.etl.q(simulation_id=sim_ind, window=window, neuron_class=nc)

    
#     plt.figure()
    # plt.hist(window_by_gid_and_trial['count'][window_by_gid_and_trial['count'] != 0], bins=[-0.1, 0.9, 1.9, 2.9])
#     plt.hist(window_by_gid_and_trial['count'], bins=[0.9, 1.9, 2.9])
#     plt.show()

    cs = np.asarray(['k', 'r', 'b', 'g', 'cyan', 'orange', 'yellow', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey', 'grey'])
    n_all_vs_firsts = []

    single_gid_images = []
    for gid in window_by_gid_and_trial.gid.unique():
#         print(gid)
        
        gid_window_by_gid_and_trial = window_by_gid_and_trial.etl.q(gid=gid)

        if np.sum(~gid_window_by_gid_and_trial["first"].isna()):

            gid_spikes = spikes_df.etl.q(gid=gid).copy()
            first_spikes_df = gid_window_by_gid_and_trial[~gid_window_by_gid_and_trial["first"].isna()]
            first_spikes_t = first_spikes_df["first"]

            n_gid_spikes = len(gid_spikes)
            n_first_spikes = len(first_spikes_t)

            n_all_vs_firsts.append([n_gid_spikes, n_first_spikes])

            do_plot = True
            if do_plot & (n_gid_spikes > plot_tresh_prop_non_first*n_first_spikes) & (n_gid_spikes > plot_tresh_n_gid_spikes):

                fig, axes = plt.subplots(2, 2, figsize=(4, 4))
                
                gid_spikes['order'] = np.concatenate([range(i) for i in gid_spikes.groupby(["trial"])["time"].count()])
                gid_spikes['c'] = cs[gid_spikes['order']]
                axes[0][0].scatter(gid_spikes.time, gid_spikes.trial, s=.5, c=gid_spikes.c)
                axes[0][1].scatter(first_spikes_t, first_spikes_df.trial, s=.5)
                axes[1][0].hist(gid_spikes.time, bins=range(0, 100, 4))
                axes[1][1].hist(first_spikes_t, bins=range(0, 100, 4))

                axes[0][0].set_xlim([0, 100])
                axes[0][1].set_xlim([0, 100])
                axes[1][0].set_xlim([0, 100])
                axes[1][1].set_xlim([0, 100])
                
                axes[0][0].set_xlabel('Time (ms)')
                axes[0][1].set_xlabel('Time (ms)')
                axes[1][0].set_xlabel('Time (ms)')
                axes[1][1].set_xlabel('Time (ms)')
                
                axes[0][0].set_ylabel('Trial')
                axes[0][1].set_ylabel('Trial')
                axes[1][0].set_ylabel('Freq')
                axes[1][1].set_ylabel('Freq')
                
                axes[0][0].set_title('All spikes')
                axes[0][1].set_title('First spikes')
                
                filename = 'temp_images/' + str(gid) + '.png'
                filenames.append(filename)
                plt.savefig(filename)
                plt.show()
                plt.close()
                
    n_all_vs_firsts = np.asarray(n_all_vs_firsts)
    plt.figure()
    plt.scatter(n_all_vs_firsts[:, 1], n_all_vs_firsts[:, 0], s=.1)
    plt.gca().set_xlim([0, 200])
    plt.gca().set_ylim([0, 200])
    plt.savefig(str(sim_ind) + '_first_vs_all.pdf')
    plt.show()
    plt.close()


    plt.figure()
    plt.hist(n_all_vs_firsts[:, 1]/n_all_vs_firsts[:, 0], np.arange(0, 0.99, 0.09))
    plt.savefig(str(sim_ind) + '_first_div_all.pdf')
    plt.show()
    plt.close()
    
c_etl.video_from_image_files(filenames, label + ".mp4", delete_images=True)
    

In [ ]:
!grep -r spikewarp ../../cortexetl.

In [ ]:
# !python ../../run.py /gpfs/bbp.cscs.ch/project/proj83/home/isbister/physiology_2023/cortexetl.configs/post_physiology_2023/campaigns/spikesorting_stimulus_tests/6_spikesorting_stimulus_test_neuropixels_17-3-24__8slc_80f_5r_100t_200ms_0.yaml